In [ ]:
import pandas as pd
import os
import shutil
import numpy as np
import re

### Dim title "MUC" -> file_name_meaning

In [2]:
df_dim = pd.read_csv("data/dimension/dim.csv")

### FUNCTION CREATED -> add key for "KEY_HOUSEHOLD"

In [ ]:
def create_key_household(dataframe):
    dataframe["key_household"] = (
        dataframe["district"].astype(str) +
        dataframe["commune"].astype(str) +
        dataframe["enumeration_area"].astype(str) +
        dataframe["household_id"].astype(str)
        )
    return dataframe

### classify MUC into each folder in raws

In [4]:
files = os.listdir("data/")
file_num = ''.join([str(num) for num in range(1,9)])
for file in files:
    if file.startswith("MUC"):
        module = file[3]
        print(module)
        if module in file_num:
            src = os.path.join("data", file)
            dst_folder = os.path.join("data/classify/raws", module)
            dst = os.path.join(dst_folder, file)
            shutil.move(src, dst)
            print(f"Moved {file} → raws/{module}")

# cleaning each folder

## FOLDER HO - HO1 - PERSONAL INFORMATION

In [5]:
file_path ="HO"
file_name ="HO1.dta"
df_data_ho1 = pd.read_stata(f"data/classify/raws/{file_path}/{file_name}")

In [6]:
variable_labels_ho1 = {
    "vung": "economic_region",
    "tinh": "province",
    "huyen": "district",
    "xa": "commune",
    "diaban": "enumeration_area",
    "hoso": "household_id",

    "quyen": "questionnaire_type",
    "tsphieu": "total_questionnaires",

    "ttnt": "area_type",
    "dantoc": "ethnicity",
    "phdich": "has_interpreter",

    "dtv": "interviewer_id",
    "dt": "team_leader_id",

    "ngaydt": "survey_day",
    "thangdt": "survey_month",
    "namdt": "survey_year",

    "tsnguoi": "household_size",

    "tinh16": "province_2016",
    "huyen16": "district_2016",
    "xa16": "commune_2016",
    "diaban16": "enumeration_area_2016",
    "ttnt16": "area_type_2016",
    "hoso16": "household_id_2016",

    "ky": "survey_wave",

    "m1b1": "has_domestic_helper_or_migrant_worker",
    "tsmuc1b": "total_domestic_helper_or_migrant_worker",

    "m1c1": "participated_vhlss_2016",
    "tsmuc1c": "total_participants_vhlss_2016",

    "m2dct": "education_total_expenditure_red_book",
    "m2xct": "education_total_expenditure_blue_book",
    "m2vct": "education_total_expenditure_yellow_book",

    "m2xtn": "education_total_income_blue",
    "m2vtn": "education_total_income_yellow",

    "m3c1g": "has_medical_treatment",

    "m3ct1": "health_expenditure_sum_q5",
    "m3ct2": "health_expenditure_sum_q6",
    "m3ct3": "health_expenditure_sum_q11",

    "m3c13": "medicine_expense",
    "m3c14": "medical_equipment_expense",
    "m3c15": "health_support_value",

    "m3ct": "total_health_expenditure",
    "m3tn": "health_income",

    "test": "test_flag",
    "loaiphieu": "questionnaire_form",

    "chuho": "household_head_name",
    "diachi": "address",

    "phone": "landline_phone",
    "mobi": "mobile_phone"
}

In [7]:
df_data_ho1_trans = df_data_ho1.rename(columns=variable_labels_ho1)

### key_household 

In [8]:
df_data_ho1_trans = create_key_household(df_data_ho1_trans)

In [9]:
df_ho1_personal_information = df_data_ho1_trans[["key_household","province","district","commune","enumeration_area","household_id","ethnicity", "household_size"]]

In [10]:
file_export = df_dim.loc[df_dim["title"] == file_name, "file_name"].values[0]
df_ho1_personal_information.to_csv(f"data/classify/cleaned/1/{file_export}",index=False)

## FOLDER 1 - MUC1 -> PERSONAL INFORMATION

In [11]:
file_path ="1"
file_name ="MUC1A.dta"
df_data_muc1a = pd.read_stata(f"data/classify/raws/{file_path}/{file_name}")
varibale_labels_muc1a = {
 'tinh': 'province',
 'huyen': 'district',
 'xa': 'commune',
 'diaban': 'enumeration_area',
 'hoso': 'household_id',

 'm1ama': 'person_code',
 'm1ac1': 'full_name',
 'm1ac2': 'gender',
 'm1ac3': 'relationship_to_head',
 'm1ac4a': 'birth_month',
 'm1ac4b': 'birth_year',
 'm1ac5': 'age',
 'm1ac6': 'has_birth_certificate',
 'm1ac7a': 'father_code',
 'm1ac7b': 'mother_code',
 'm1ac7c': 'spouse_code',
 'm1ac8': 'marital_status',
 'm1ac9': 'months_in_household',
 'm1ac10': 'reason_not_living_in_household',
 'm1ama1': 'person_code_alt',
 'm1ac11': 'birth_province',
 'm1ac12': 'household_registration_status',
 'm1ac13': 'household_registration_province',
 'm1ac14a': 'years_in_province',
 'm1ac14b': 'months_in_province',
 'm1ac15a': 'member_farmers_union',
 'm1ac15b': 'member_womens_union',
 'm1ac15c': 'member_communist_party',
 'm1ac15d': 'member_veterans_union',
 'm1ac16': 'used_internet_last_30_days'
}

In [12]:
df_data_muc1a_trans = df_data_muc1a.rename(columns=varibale_labels_muc1a)

In [13]:
df_personal_information = df_data_muc1a_trans.copy()

In [14]:
df_personal_information["key_household"] = (
    df_personal_information["district"].astype(str) +
    df_personal_information["commune"].astype(str) +
    df_personal_information["enumeration_area"].astype(str) +
    df_personal_information["household_id"].astype(str)
)
df_personal_information_filtered = df_personal_information[["key_household","province", "district", "commune","enumeration_area", "household_id", "person_code", "gender", "relationship_to_head", "age", "marital_status"]]

### MOTHER PROXY RECOGNITION

In [ ]:
df_motherhood = df_personal_information_filtered.copy()
df_motherhood["has_grandchild"] = (
    df_motherhood.groupby("key_household")["relationship_to_head"]
    .transform(lambda x: (x == "Cháu nội ngoại").any())
)

df_motherhood["has_child"] = (
    df_motherhood.groupby("key_household")["relationship_to_head"]
    .transform(lambda x: (x == "Con").any())
)

df_motherhood["female_head"] = (
    (df_motherhood["relationship_to_head"] == "Chủ hộ") &
    (df_motherhood["gender"] == "Nữ")
)

df_motherhood["not_related_direct_to_head"] = (
    (df_motherhood["relationship_to_head"] == "Con") &
    (df_motherhood["gender"] == "Nữ")
)

df_motherhood["wifey"] = (
    (df_motherhood["relationship_to_head"] == "Vợ/chồng") &
    (df_motherhood["gender"] == "Nữ")
)
df_motherhood["mother_proxy"] = (
    (df_motherhood["has_grandchild"] & df_motherhood["not_related_direct_to_head"]) |
    (df_motherhood["has_child"] & (df_motherhood["female_head"] | df_motherhood["wifey"]))
)

### MOTHER PROXY HAS CHILD < 6 YEARS OLD

In [16]:
df_children_below_6 = df_motherhood.copy()
df_children_below_6["is_child_under6"] = (
    (df_children_below_6["relationship_to_head"] == "Con") &
    (df_children_below_6["age"] < 6)
)

df_children_below_6["is_grandchild_under6"] = (
    (df_children_below_6["relationship_to_head"] == "Cháu nội ngoại") &
    (df_children_below_6["age"] < 6)
)

df_children_below_6["has_child_below_6"] = (
    df_children_below_6.groupby("key_household")["is_child_under6"].transform("any")
)

df_children_below_6["has_grandchild_below_6"] = (
    df_children_below_6.groupby("key_household")["is_grandchild_under6"].transform("any")
)
df_children_below_6["mother_proxy_child_below_6"] = (
    (df_children_below_6["mother_proxy"] & df_children_below_6["has_child_below_6"]) |
    (df_children_below_6["mother_proxy"] & df_children_below_6["has_grandchild_below_6"])
)


In [17]:
df_children_below_6["age_square"] = df_children_below_6["age"]**2

### allocate to folder classify/cleaned/1

In [18]:
file_export = df_dim.loc[df_dim["title"] == file_name, "file_name"].values[0]

df_children_below_6.to_csv(f"data/classify/cleaned/1/{file_export}",index=False)

## FOLDER 4 - MUC4 -> EMPLOYMENT

In [19]:
file_path ="4"
file_name ="MUC4A.dta"
df_data_muc4a = pd.read_stata(f"data/classify/raws/{file_path}/{file_name}")
varibale_labels_muc4a = {
    "tinh": "province",
    "huyen": "district",
    "xa": "commune",
    "diaban": "enumeration_area",
    "hoso": "household_id",

    # person key
    "m4ama": "person_code",
    "m4ama1": "person_code_1",
    "m4ama2": "person_code_2",
    "m4ama3": "person_code_3",
    "m4ama4": "person_code_4",

    # employment status
    "m4ac1a": "wage_employment",
    "m4ac1b": "self_employed_agriculture",
    "m4ac1c": "self_employed_business_service",
    "m4ac2": "has_job",
    "m4ac2a": "reason_not_working",

    # primary job
    "m4ac3a": "working_days",
    "m4ac3": "occupation_code",
    "m4ac4": "industry_code",
    "m4ac5": "worked_last_30_days",
    "m4ac6": "number_of_days_worked",
    "m4ac7": "working_hours",

    # job characteristics
    "m4ac8a": "economic_sector_type",
    "m4ac8b": "is_civil_servant",

    # income primary job
    "m4ac9": "has_salary",
    "m4ac10": "salary_last_30_days",
    "m4ac11": "salary_last_12_months",
    "m4ac12a": "holiday_bonus",
    "m4ac12b": "other_benefits",

    # contract & benefits
    "m4ac13a": "has_labor_contract",
    "m4ac13b": "paid_leave",
    "m4ac13c": "social_insurance",

    # commuting
    "m4ac14": "transport_to_work",
    "m4ac15": "transport_cost",
    "m4ac16": "travel_time_minutes",

    # overtime / secondary job
    "m4ac17": "has_overtime_last_30_days",
    "m4ac18a": "overtime_days",
    "m4ac18": "secondary_job_occupation_code",
    "m4ac19": "secondary_job_industry_code",
    "m4ac20": "worked_secondary_job_last_30_days",

    # secondary job workload
    "m4ac21": "secondary_job_working_days",
    "m4ac22": "secondary_job_avg_hours_per_day",

    # secondary job income
    "m4ac23": "worked_for_org_or_individual",
    "m4ac24": "received_salary_secondary_job",
    "m4ac25": "secondary_job_income_last_30_days",
    "m4ac26": "secondary_job_income_last_12_months",
    "m4ac27a": "secondary_job_holiday_bonus",
    "m4ac27b": "secondary_job_other_benefits",

    # other jobs
    "m4ac28": "has_other_paid_job",
    "m4ac29": "other_job_income",

    # unemployment benefits
    "m4ac30": "received_unemployment_benefit_12_months",
    "m4ac31a": "unemployment_benefit",
    "m4ac31b": "severance_pay",
    "m4ac31c": "regular_pension",
    "m4ac31d": "early_retirement_pension",
    "m4ac31e": "disability_allowance",

    # descriptions
    "m4ac3m": "job_description",
    "m4ac4c": "organization_name",
    "m4ac4m": "main_product_service_description",
    "m4ac18m": "secondary_job_description",
    "m4ac19a": "secondary_org_name",
    "m4ac19m": "secondary_product_service_description"
}

In [20]:
df_employment = df_data_muc4a.rename(columns=varibale_labels_muc4a)

In [21]:
df_employment["key_household"] = (
    df_employment["district"].astype(str) +
    df_employment["commune"].astype(str) +
    df_employment["enumeration_area"].astype(str) +
    df_employment["household_id"].astype(str)
)

In [22]:
numeric_columns = ["salary_last_12_months","holiday_bonus","other_benefits","secondary_job_income_last_12_months","secondary_job_holiday_bonus","secondary_job_other_benefits","other_job_income","unemployment_benefit","severance_pay","regular_pension", "early_retirement_pension","disability_allowance"]
id_columns = ["key_household","province","district","commune","enumeration_area","household_id","person_code","has_job","working_hours"]
df_employment_filtered = df_employment[id_columns + numeric_columns]

### change datatype for numeric columns

In [23]:
for col in numeric_columns:
    df_employment_filtered[col] = df_employment_filtered[col].astype(float)
    df_employment_filtered[col] = df_employment_filtered[col].fillna(0)

### addding total_income column

In [24]:
df_employment_filtered["total_income"] = df_employment_filtered[numeric_columns].sum(axis=1)

### allocate to folder classify/cleaned/4

In [25]:
file_export = df_dim.loc[df_dim["title"] == file_name, "file_name"].values[0]
df_employment_filtered.to_csv(f"data/classify/cleaned/4/{file_export}",index=False)

## FOLDER 2 -> MUC2 -> EDUCATION

In [26]:
file_path ="2"
file_name ="MUC2AB_CLEANED_INEQ.dta"
df_data_muc2 = pd.read_stata(f"data/classify/raws/{file_path}/{file_name}")

In [27]:
variable_labels_muc2 = {
    "tinh": "province",
    "huyen": "district",
    "xa": "commune",
    "diaban": "enumeration_area",
    "hoso": "household_id",
    "matv": "person_code",

    "m2vc1": "education_highest_grade_completed",
    "m2vc2a": "highest_degree_general_education",
    "m2vc2b": "highest_degree_vocational_training",
    "m2vc3": "school_type",
    "m2vc4": "currently_attending_school",
    "m2vc5": "attended_school_last_12_months",
    "m2vc6": "education_level_current",
    "m2vc7": "current_grade",
    "m2vc8": "school_type_current",

    "m2vma1": "person_code_1",

    "m2vc9": "tuition_fee_exemption",
    "m2vc10a": "reason_tuition_exemption",
    "m2vc10b": "reason_contribution_exemption",
    "m2vc10a1": "tuition_exemption_percentage",
    "m2vc10a2": "contribution_exemption_percentage",

    "m2vc11a": "education_cost_tuition",
    "m2vc11b": "education_cost_out_of_zone",
    "m2vc11c": "education_cost_contribution",
    "m2vc11d": "education_cost_fund",
    "m2vc11e": "education_cost_uniform",
    "m2vc11f": "education_cost_textbooks",

    "m2vc11g": "education_cost_supplies",
    "m2vc11g1": "education_cost_notebooks",
    "m2vc11g2": "education_cost_stationery",
    "m2vc11g3": "education_cost_devices",

    "m2vma2": "person_code_2",

    "m2vc11h": "education_cost_extra_classes",
    "m2vc11i": "education_cost_other",
    "m2vc11i1": "education_cost_student_housing",

    "m2vc11k": "education_cost_total",

    "m2vc12": "transport_to_school",
    "m2vc13": "transport_cost",
    "m2vc14": "travel_time_minutes",

    "m2vc15": "education_support_received",
    "m2vc16": "scholarship_value",

    "m2vma3": "person_code_3",

    "m2vc17": "other_training_cost",
    "m2vc18a": "toy_purchase_outside",
    "m2vc18b": "toy_self_made",

    "m2vc19": "household_books_count",

    "dup": "duplicate_or_not"
}

In [28]:
df_education = df_data_muc2.rename(columns=variable_labels_muc2)

In [29]:
df_education["key_household"] = (
    df_education["district"].astype(str) +
    df_education["commune"].astype(str) +
    df_education["enumeration_area"].astype(str) +
    df_education["household_id"].astype(str)
)

In [30]:
df_education_filtered = df_education[["key_household","province","district","commune","enumeration_area","household_id","person_code","education_highest_grade_completed", "highest_degree_general_education", "highest_degree_vocational_training","education_level_current","current_grade","school_type_current"]]

In [31]:
# transform grade
df_education_filtered["education_highest_grade_completed"].unique()
# mismatch in data output
# ['00',  '4',  '9',  '5',  '3',  '2', '12',  '1',  '8', '11',  '6',  '7',  '0',
#  '03', '09', '10', '01', '04', '02', '05', '08', '06', '07', '8.', '.4', '9+',
#  '.3', '9.', '3.', '4.', '0.', '.0', '.8']

def normalize_var(val):
    if pd.isna(val):
        return np.na
    
    val = str(val).strip()
    if val =="00":
        return val
    
    # special
    if val.endswith("+"):
        return str(val[:-1])

    match = re.search(r"\d+", val)
    if match:
        return str(match.group())
    
    return np.nan
    
# transform and normalize
df_education_filtered["education_highest_grade_completed"] = df_education_filtered["education_highest_grade_completed"].apply(normalize_var)


### allocate to folder classify/cleaned/2

In [32]:
file_export = df_dim.loc[df_dim["title"] == file_name, "file_name"].values[0]
df_education_filtered.to_csv(f"data/classify/cleaned/2/{file_export}",index=False)

## FOLDER 5 -> MUC5/HO4 -> total_cost HO

In [33]:
file_path ="HO"
file_name ="HO4.dta"
df_data_ho4 = pd.read_stata(f"data/classify/raws/{file_path}/{file_name}")

In [34]:
variable_labels_ho4 = {
   "tinh": "province",
    "huyen": "district",
    "xa": "commune",
    "diaban": "enumeration_area",
    "hoso": "household_id",

    "m5a1ct": "total_cost_yearly_food_and_beverages_expenditures_during_holidays_and_fesitval",
    "m5a1c4": "food_expenditure_sum_q2b",
    "m5a1c5": "food_expenditure_sum_q3b",

    "m5a2ct": "total_cost_monthly_regular_food_and_beverages_expenditures",
    "m5a2c6": "food_expenditure_sum_q3b",
    "m5a2c7": "food_expenditure_sum_q4b",
    "m5a2c8": "food_expenditure_sum_q5b",

    "m5b1ct": "total_cost_monthly_for_daily_spending_exclude_food",
    "m5b1c6": "nonfood_expenditure_sum_q3",
    "m5b1c7": "nonfood_expenditure_sum_q4",
    "m5b1c8": "nonfood_expenditure_sum_q5",

    "m5b2ct": "total_cost_yearly_spending_exclude_food",
    "m5b2c4": "durable_goods_expenditure_sum_q2",
    "m5b2c5": "durable_goods_expenditure_sum_q3",

    "m5b3ct": "total_cost_others_expenditures_included_in_total_expenditure",

    "m6bc7": "household_item_purchase_total"
}

In [35]:
df_data_ho4_trans = df_data_ho4.rename(columns=variable_labels_ho4)

In [36]:
df_data_ho4_trans = create_key_household(df_data_ho4_trans)

In [37]:
df_cost_filtered = df_data_ho4_trans[["key_household","province","district","commune","enumeration_area","household_id","total_cost_yearly_food_and_beverages_expenditures_during_holidays_and_fesitval","total_cost_monthly_regular_food_and_beverages_expenditures","total_cost_monthly_for_daily_spending_exclude_food","total_cost_yearly_spending_exclude_food","total_cost_others_expenditures_included_in_total_expenditure"]]

### allocate to folder classify/cleaned/HO

In [38]:
file_export = df_dim.loc[df_dim["title"] == file_name, "file_name"].values[0]
df_cost_filtered.to_csv(f"data/classify/cleaned/5/{file_export}",index=False)